In [44]:
import numpy as np
import pandas as pd
import bm25s
from pathlib import Path
import random

In [2]:
data_path = Path('data/')

collection_data_path = data_path / 'subtask4b_collection_data.pkl' 
query_dev_data_path = data_path / 'subtask4b_query_tweets_dev.tsv'
query_train_data_path = data_path / 'subtask4b_query_tweets_train.tsv'

In [3]:
df_collection = pd.read_pickle(collection_data_path)
df_collection.head(5)

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


In [4]:
df_query_dev = pd.read_csv(query_dev_data_path, sep='\t')
df_query_train = pd.read_csv(query_train_data_path, sep='\t')
display(df_query_dev)
display(df_query_train)

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy
...,...,...,...
1395,14193,Residents at high risk of covid-19: effectiven...,0gn3b98n
1396,14196,"61% of teenagers hospitalized for covid were ""...",25bdifv6
1397,14203,"""fresh evidence backing melatonin against covi...",qn6wawxk
1398,14233,"the vaccine doesn't halt the spread, it is pro...",3u3i5myh


,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69
...,...,...,...
12848,14248,"""evidence on covid-19 reveals a growing body o...",9169o29b
12849,14249,Outdoor lighting has detrimental impacts on lo...,s2bpha8l
12850,14250,"26/ and influenza virus (and other pathogens, ...",atloc9th
12851,14251,does it?'sars-cov-2-naïve vaccinees had a 13.0...,t4y1ylb3


In [5]:
df_collection['last_names'] = df_collection['authors'].str.split(';').apply(
    lambda authors: [author.split(',')[0].strip() for author in authors] if isinstance(authors, list) else authors
)
df_collection['publish_year'] = df_collection['publish_time'].str.split('-').str[0]

In [6]:
def initialize_bm25(corpus: list[str], cord_uids:list[str], k1=1.5, b=0.75, method='lucene'):
    tokenized_corpus = bm25s.tokenize(corpus)
    bm25 = bm25s.BM25(corpus=cord_uids, k1=k1, b=b, method=method)
    bm25.index(tokenized_corpus)
    return bm25

In [7]:
def experiment_single_bm25(df_collection, df_query, k1=1.5, b=0.75):
    corpus = df_collection.apply(
        lambda x: f"{x['title']} {x['abstract']} {x['last_names']} {x['journal']} {x['publish_year']}", axis=1
    ).tolist()
    bm25 = initialize_bm25(corpus, df_collection['cord_uid'].tolist(), k1, b)
    tokenized_queries = bm25s.tokenize(df_query['tweet_text'])
    doc_scores = bm25.retrieve(tokenized_queries, n_threads=-1)
    df_query['bm25_topk'] = doc_scores.documents.tolist()
    
    return df_query

In [30]:
def experiment_ensemble_single_feature(df_collection, df_query, k1=1.5, b=0.75, weights=None):
    features = ['title', 'abstract', 'last_names', 'journal', 'publish_year']
    bm25_models = {}
    
    if weights is None:
        weights = {feature: 1.0 for feature in features}
    
    for feature in features:
        corpus = df_collection.apply(
            lambda x: f"{x[feature]}", axis=1
        ).tolist()
        bm25_models[feature] = initialize_bm25(corpus, df_collection['cord_uid'].tolist(), k1, b)
    
    tokenized_queries = bm25s.tokenize(df_query['tweet_text'])
    
    all_scores = {feature: bm25.retrieve(tokenized_queries, n_threads=-1, k=200) for feature, bm25 in bm25_models.items()}
    
    ensemble_scores = []
    for query_idx in range(len(df_query)):
        scores = {cord_uid: 0 for cord_uid in df_collection['cord_uid']}
        for feature, doc_scores in all_scores.items():
            for cord_uid, score in zip(doc_scores.documents[query_idx], doc_scores.scores[query_idx]):
                scores[cord_uid] += weights[feature] * score
        sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        ensemble_scores.append([cord_uid for cord_uid, _ in sorted_scores[:10]])
    
    df_query['bm25_topk'] = ensemble_scores
    return df_query

In [39]:
def experiment_ensemble_all_features(df_collection, df_query, param_sets):
    bm25_models = {}
    corpus = df_collection.apply(
        lambda x: f"{x['title']} {x['abstract']} {x['last_names']} {x['journal']} {x['publish_year']}", axis=1
    ).tolist()
    for param in param_sets:
        if 'method' in param:
            method = param['method']
        else:
            method = "AHHH no method??"
        bm25_models[method] = initialize_bm25(corpus, df_collection['cord_uid'].tolist(), **param)

    tokenized_queries = bm25s.tokenize(df_query['tweet_text'])

    all_scores = {model: bm25.retrieve(tokenized_queries, n_threads=-1, k=100) for model, bm25 in bm25_models.items()}
    
    ensemble_scores = []
    for query_idx in range(len(df_query)):
        scores = {cord_uid: 0 for cord_uid in df_collection['cord_uid']}
        for feature, doc_scores in all_scores.items():
            for cord_uid, score in zip(doc_scores.documents[query_idx], doc_scores.scores[query_idx]):
                scores[cord_uid] += score
        sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        ensemble_scores.append([cord_uid for cord_uid, _ in sorted_scores[:10]])

    df_query['bm25_topk'] = ensemble_scores
    return df_query

In [11]:
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [32]:
def evaluate_experiment(df_query_train, df_query_dev, experiment_name):
    results = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
    print(f"Results for {experiment_name}, train: {results}")
    results = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')
    print(f"Results for {experiment_name}, dev: {results}")
    return results

In [33]:
# Experiment 1: Single BM25 with all features
df_query_train_single = experiment_single_bm25(df_collection, df_query_train)
df_query_dev_single = experiment_single_bm25(df_collection, df_query_dev)
evaluate_experiment(df_query_train_single, df_query_dev_single, "Single BM25 with all features")

Results for Single BM25 with all features, train: {1: np.float64(0.586711273632615), 5: np.float64(0.6348012137244223), 10: np.float64(0.6403489272469277)}
Results for Single BM25 with all features, dev: {1: np.float64(0.5928571428571429), 5: np.float64(0.6401190476190476), 10: np.float64(0.6454557823129252)}


{1: np.float64(0.5928571428571429),
 5: np.float64(0.6401190476190476),
 10: np.float64(0.6454557823129252)}

In [34]:
weights = {
    'title': 1.0,
    'abstract': 1.0,
    'last_names': 1.0,
    'journal': 1.0,
    'publish_year': 1.0
}

# Experiment 2: Ensemble of BM25 models, each trained on one feature
df_query_train_ensemble_single = experiment_ensemble_single_feature(df_collection, df_query_train, weights=weights)
df_query_dev_ensemble_single = experiment_ensemble_single_feature(df_collection, df_query_dev, weights=weights)
evaluate_experiment(df_query_train_ensemble_single, df_query_dev_ensemble_single, "Ensemble BM25 with single features")

Results for Ensemble BM25 with single features, train: {1: np.float64(0.5687388158406598), 5: np.float64(0.6185144842967919), 10: np.float64(0.6240079828191553)}
Results for Ensemble BM25 with single features, dev: {1: np.float64(0.5707142857142857), 5: np.float64(0.6254404761904762), 10: np.float64(0.6302525510204081)}


{1: np.float64(0.5707142857142857),
 5: np.float64(0.6254404761904762),
 10: np.float64(0.6302525510204081)}

In [37]:
weights = {
    'title': 1.0,
    'abstract': 1.0,
    'last_names': 0.5,
    'journal': 0.2,
    'publish_year': 0.05
}
df_query_train_ensemble_single = experiment_ensemble_single_feature(df_collection, df_query_train, weights=weights)
df_query_dev_ensemble_single = experiment_ensemble_single_feature(df_collection, df_query_dev, weights=weights)
evaluate_experiment(df_query_train_ensemble_single, df_query_dev_ensemble_single, "Ensemble BM25 with single features")

weights = {
    'title': 1.0,
    'abstract': 1.0,
    'last_names': 1.0,
    'journal': 0.7,
    'publish_year': 0.05
}
df_query_train_ensemble_single = experiment_ensemble_single_feature(df_collection, df_query_train, weights=weights)
df_query_dev_ensemble_single = experiment_ensemble_single_feature(df_collection, df_query_dev, weights=weights)
evaluate_experiment(df_query_train_ensemble_single,df_query_dev_ensemble_single, "Ensemble BM25 with single features")

Results for Ensemble BM25 with single features, train: {1: np.float64(0.5706060841826811), 5: np.float64(0.6195337016001453), 10: np.float64(0.6250072554242787)}
Results for Ensemble BM25 with single features, dev: {1: np.float64(0.5735714285714286), 5: np.float64(0.6275476190476191), 10: np.float64(0.63234268707483)}


Results for Ensemble BM25 with single features, train: {1: np.float64(0.5693612386213336), 5: np.float64(0.6186999144168677), 10: np.float64(0.624236729365882)}
Results for Ensemble BM25 with single features, dev: {1: np.float64(0.5721428571428572), 5: np.float64(0.6266071428571428), 10: np.float64(0.6315274943310658)}


{1: np.float64(0.5721428571428572),
 5: np.float64(0.6266071428571428),
 10: np.float64(0.6315274943310658)}

In [42]:
param_sets = [
    {'method': 'robertson', 'k1': 2.0, 'b': 0.6},
    {'method': 'lucene', 'k1': 2.0, 'b': 0.6},
    {'method': 'atire', 'k1': 2.0, 'b': 0.6},
    {'method': 'bm25l', 'k1': 2.0, 'b': 0.6},
    {'method': 'bm25+', 'k1': 2.0, 'b': 0.6}
]

# Experiment 3: Ensemble of slightly different BM25 models, all trained on all features
df_query_train_ensemble_all = experiment_ensemble_all_features(df_collection, df_query_train, param_sets)
df_query_dev_ensemble_all = experiment_ensemble_all_features(df_collection, df_query_dev, param_sets)
evaluate_experiment(df_query_train_ensemble_all, df_query_dev_ensemble_all, "Ensemble BM25 with all features and different models")

Results for Ensemble BM25 with all features and different models, train: {1: np.float64(0.5781529603983506), 5: np.float64(0.6287624160377603), 10: np.float64(0.6342932105283308)}
Results for Ensemble BM25 with all features and different models, dev: {1: np.float64(0.5857142857142857), 5: np.float64(0.6357142857142857), 10: np.float64(0.641640873015873)}


{1: np.float64(0.5857142857142857),
 5: np.float64(0.6357142857142857),
 10: np.float64(0.641640873015873)}

In [50]:
methods = ['robertson', 'lucene', 'atire', 'bm25l', 'bm25+']
param_sets = []
for _ in range(20):
    k1 = round(random.uniform(1.0, 3.0), 2)  # Random k1 between 1.0 and 3.0
    b = round(random.uniform(0.0, 1.0), 2)   # Random b between 0.0 and 1.0
    param_set = []
    for method in methods:
        param_set.append({'method': method, 'k1': k1, 'b': b})
    param_sets.append(param_set)

In [52]:
for param_set in param_sets:
    df_query_train_ensemble_all = experiment_ensemble_all_features(df_collection, df_query_train, param_set)
    df_query_dev_ensemble_all = experiment_ensemble_all_features(df_collection, df_query_dev, param_set)
    evaluate_experiment(df_query_train_ensemble_all, df_query_dev_ensemble_all, 
        f"Ensemble BM25 with all features and different models, k1={param_set[0]['k1']}, b={param_set[0]['b']}")

Results for Ensemble BM25 with all features and different models, k1=1.7, b=0.07, train: {1: np.float64(0.5532560491713997), 5: np.float64(0.6043478824658315), 10: np.float64(0.6099020795589689)}
Results for Ensemble BM25 with all features and different models, k1=1.7, b=0.07, dev: {1: np.float64(0.5635714285714286), 5: np.float64(0.6170833333333334), 10: np.float64(0.6226921768707483)}


Results for Ensemble BM25 with all features and different models, k1=1.12, b=0.27, train: {1: np.float64(0.573017972457792), 5: np.float64(0.6208770974350993), 10: np.float64(0.6261461717911574)}
Results for Ensemble BM25 with all features and different models, k1=1.12, b=0.27, dev: {1: np.float64(0.5864285714285714), 5: np.float64(0.6324166666666666), 10: np.float64(0.6383531746031745)}


Results for Ensemble BM25 with all features and different models, k1=1.8, b=0.26, train: {1: np.float64(0.5680385902124018), 5: np.float64(0.6169454602038434), 10: np.float64(0.6226811046522398)}
Results for Ensemble BM25 with all features and different models, k1=1.8, b=0.26, dev: {1: np.float64(0.5828571428571429), 5: np.float64(0.6305833333333333), 10: np.float64(0.6364951814058957)}


Results for Ensemble BM25 with all features and different models, k1=2.01, b=0.84, train: {1: np.float64(0.5819652999299775), 5: np.float64(0.6318265515184522), 10: np.float64(0.6377257363175047)}
Results for Ensemble BM25 with all features and different models, k1=2.01, b=0.84, dev: {1: np.float64(0.5871428571428572), 5: np.float64(0.6364047619047619), 10: np.float64(0.6417165532879819)}


Results for Ensemble BM25 with all features and different models, k1=1.12, b=0.9, train: {1: np.float64(0.5864778650898623), 5: np.float64(0.6346041131772089), 10: np.float64(0.6396156354084465)}
Results for Ensemble BM25 with all features and different models, k1=1.12, b=0.9, dev: {1: np.float64(0.5885714285714285), 5: np.float64(0.6372857142857143), 10: np.float64(0.6431935941043083)}


...

Results for Ensemble BM25 with all features and different models, k1=1.04, b=0.35, train: {1: np.float64(0.5760522835135766), 5: np.float64(0.6240358930470189), 10: np.float64(0.6292141974142285)}
Results for Ensemble BM25 with all features and different models, k1=1.04, b=0.35, dev: {1: np.float64(0.5878571428571429), 5: np.float64(0.6334642857142857), 10: np.float64(0.6393840702947845)}


Results for Ensemble BM25 with all features and different models, k1=1.44, b=0.89, train: {1: np.float64(0.5853886252236832), 5: np.float64(0.6343966389169843), 10: np.float64(0.6398440732137145)}
Results for Ensemble BM25 with all features and different models, k1=1.44, b=0.89, dev: {1: np.float64(0.5857142857142857), 5: np.float64(0.6375714285714286), 10: np.float64(0.6426397392290248)}


Results for Ensemble BM25 with all features and different models, k1=1.25, b=0.1, train: {1: np.float64(0.5615031510153271), 5: np.float64(0.6105293187063979), 10: np.float64(0.6158670262887177)}
Results for Ensemble BM25 with all features and different models, k1=1.25, b=0.1, dev: {1: np.float64(0.5757142857142857), 5: np.float64(0.623202380952381), 10: np.float64(0.629829081632653)}


Results for Ensemble BM25 with all features and different models, k1=2.62, b=0.3, train: {1: np.float64(0.5622811794911694), 5: np.float64(0.6131772089525143), 10: np.float64(0.6191934993868394)}
Results for Ensemble BM25 with all features and different models, k1=2.62, b=0.3, dev: {1: np.float64(0.5721428571428572), 5: np.float64(0.6246904761904761), 10: np.float64(0.6301173469387756)}


Results for Ensemble BM25 with all features and different models, k1=2.31, b=0.66, train: {1: np.float64(0.5770637205321715), 5: np.float64(0.628273554812106), 10: np.float64(0.6340757330448453)}
Results for Ensemble BM25 with all features and different models, k1=2.31, b=0.66, dev: {1: np.float64(0.5814285714285714), 5: np.float64(0.6335119047619048), 10: np.float64(0.639033163265306)}


In [53]:
methods = ['robertson', 'lucene', 'atire', 'bm25l', 'bm25+']
param_sets = []
for _ in range(20):
    k1 = round(random.uniform(1.0, 3.0), 2)  # Random k1 between 1.0 and 3.0
    b = round(random.uniform(0.7, 1.2), 2)   # Random b between 0.0 and 1.0
    param_set = []
    for method in methods:
        param_set.append({'method': method, 'k1': k1, 'b': b})
    param_sets.append(param_set)

In [54]:
for param_set in param_sets:
    df_query_train_ensemble_all = experiment_ensemble_all_features(df_collection, df_query_train, param_set)
    df_query_dev_ensemble_all = experiment_ensemble_all_features(df_collection, df_query_dev, param_set)
    evaluate_experiment(df_query_train_ensemble_all, df_query_dev_ensemble_all, 
        f"Ensemble BM25 with all features and different models, k1={param_set[0]['k1']}, b={param_set[0]['b']}")

Results for Ensemble BM25 with all features and different models, k1=1.81, b=1.16, train: {1: np.float64(0.5669493503462226), 5: np.float64(0.6203908296376981), 10: np.float64(0.6263075818257487)}
Results for Ensemble BM25 with all features and different models, k1=1.81, b=1.16, dev: {1: np.float64(0.57), 5: np.float64(0.6223809523809524), 10: np.float64(0.6289135487528346)}


Results for Ensemble BM25 with all features and different models, k1=2.03, b=0.82, train: {1: np.float64(0.5818874970823932), 5: np.float64(0.6319043543660364), 10: np.float64(0.6377702259616987)}
Results for Ensemble BM25 with all features and different models, k1=2.03, b=0.82, dev: {1: np.float64(0.585), 5: np.float64(0.6352619047619048), 10: np.float64(0.6402859977324263)}


Results for Ensemble BM25 with all features and different models, k1=1.05, b=0.87, train: {1: np.float64(0.5863222593946938), 5: np.float64(0.6342838247879872), 10: np.float64(0.6394099518486821)}
Results for Ensemble BM25 with all features and different models, k1=1.05, b=0.87, dev: {1: np.float64(0.59), 5: np.float64(0.6377619047619046), 10: np.float64(0.6435399659863945)}


Results for Ensemble BM25 with all features and different models, k1=1.43, b=1.12, train: {1: np.float64(0.578308566093519), 5: np.float64(0.6286859099043025), 10: np.float64(0.6344206898766146)}
Results for Ensemble BM25 with all features and different models, k1=1.43, b=1.12, dev: {1: np.float64(0.5764285714285714), 5: np.float64(0.6298333333333334), 10: np.float64(0.6354172335600906)}


Results for Ensemble BM25 with all features and different models, k1=1.22, b=1.1, train: {1: np.float64(0.5822765113203143), 5: np.float64(0.6314647682771857), 10: np.float64(0.6369693814920611)}
Results for Ensemble BM25 with all features and different models, k1=1.22, b=1.1, dev: {1: np.float64(0.5792857142857143), 5: np.float64(0.632702380952381), 10: np.float64(0.6382712585034014)}


...

Results for Ensemble BM25 with all features and different models, k1=1.53, b=1.09, train: {1: np.float64(0.5796312145024508), 5: np.float64(0.6295871262221531), 10: np.float64(0.6353260433300113)}
Results for Ensemble BM25 with all features and different models, k1=1.53, b=1.09, dev: {1: np.float64(0.5764285714285714), 5: np.float64(0.630047619047619), 10: np.float64(0.6353747165532879)}


Results for Ensemble BM25 with all features and different models, k1=2.04, b=1.1, train: {1: np.float64(0.5699058585544231), 5: np.float64(0.6226172877927332), 10: np.float64(0.6286082923015934)}
Results for Ensemble BM25 with all features and different models, k1=2.04, b=1.1, dev: {1: np.float64(0.5714285714285714), 5: np.float64(0.6234642857142857), 10: np.float64(0.6306139455782312)}


Results for Ensemble BM25 with all features and different models, k1=1.59, b=0.84, train: {1: np.float64(0.5855442309188517), 5: np.float64(0.6345794756088072), 10: np.float64(0.6401674798916689)}
Results for Ensemble BM25 with all features and different models, k1=1.59, b=0.84, dev: {1: np.float64(0.5871428571428572), 5: np.float64(0.6374642857142857), 10: np.float64(0.6423696145124717)}


Results for Ensemble BM25 with all features and different models, k1=2.41, b=1.09, train: {1: np.float64(0.5644596592235276), 5: np.float64(0.6187401125547862), 10: np.float64(0.6247427566166381)}
Results for Ensemble BM25 with all features and different models, k1=2.41, b=1.09, dev: {1: np.float64(0.5678571428571428), 5: np.float64(0.6208095238095238), 10: np.float64(0.6277565192743764)}


Results for Ensemble BM25 with all features and different models, k1=1.49, b=0.71, train: {1: np.float64(0.5846883995954252), 5: np.float64(0.6333475971887237), 10: np.float64(0.6389197630347556)}
Results for Ensemble BM25 with all features and different models, k1=1.49, b=0.71, dev: {1: np.float64(0.59), 5: np.float64(0.6384285714285713), 10: np.float64(0.6441303854875283)}


In [56]:
def experiment_ensemble_some_features(df_collection, df_query, param_sets):
    bm25_models = {}
    corpus = df_collection.apply(
        lambda x: f"{x['title']} {x['abstract']}", axis=1
    ).tolist()
    for param in param_sets:
        if 'method' in param:
            method = param['method']
        else:
            method = "AHHH no method??"
        bm25_models[method] = initialize_bm25(corpus, df_collection['cord_uid'].tolist(), **param)

    tokenized_queries = bm25s.tokenize(df_query['tweet_text'])

    all_scores = {model: bm25.retrieve(tokenized_queries, n_threads=-1, k=100) for model, bm25 in bm25_models.items()}
    
    ensemble_scores = []
    for query_idx in range(len(df_query)):
        scores = {cord_uid: 0 for cord_uid in df_collection['cord_uid']}
        for feature, doc_scores in all_scores.items():
            for cord_uid, score in zip(doc_scores.documents[query_idx], doc_scores.scores[query_idx]):
                scores[cord_uid] += score
        sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        ensemble_scores.append([cord_uid for cord_uid, _ in sorted_scores[:10]])

    df_query['bm25_topk'] = ensemble_scores
    return df_query

In [57]:
methods = ['robertson', 'lucene', 'atire', 'bm25l', 'bm25+']
param_sets = []
for _ in range(5):
    k1 = round(random.uniform(1.0, 3.0), 2)  
    b = round(random.uniform(0.65, 0.85), 2)  
    param_set = []
    for method in methods:
        param_set.append({'method': method, 'k1': k1, 'b': b})
    param_sets.append(param_set)

In [58]:
for param_set in param_sets:
    df_query_train_ensemble_some = experiment_ensemble_some_features(df_collection, df_query_train, param_set)
    df_query_dev_ensemble_some = experiment_ensemble_some_features(df_collection, df_query_dev, param_set)
    evaluate_experiment(df_query_train_ensemble_some, df_query_dev_ensemble_some, 
        f"Ensemble BM25 with all features and different models, k1={param_set[0]['k1']}, b={param_set[0]['b']}")

Results for Ensemble BM25 with all features and different models, k1=1.39, b=0.82, train: {1: np.float64(0.5814984828444721), 5: np.float64(0.6309434891983713), 10: np.float64(0.6361636589073763)}
Results for Ensemble BM25 with all features and different models, k1=1.39, b=0.82, dev: {1: np.float64(0.5871428571428572), 5: np.float64(0.6380476190476191), 10: np.float64(0.643187074829932)}


Results for Ensemble BM25 with all features and different models, k1=1.1, b=0.77, train: {1: np.float64(0.5821987084727301), 5: np.float64(0.6301952851474364), 10: np.float64(0.6354060999902438)}
Results for Ensemble BM25 with all features and different models, k1=1.1, b=0.77, dev: {1: np.float64(0.59), 5: np.float64(0.6370119047619048), 10: np.float64(0.6430609410430839)}


Results for Ensemble BM25 with all features and different models, k1=2.72, b=0.8, train: {1: np.float64(0.5717731268964444), 5: np.float64(0.6244300941414456), 10: np.float64(0.629892070160879)}
Results for Ensemble BM25 with all features and different models, k1=2.72, b=0.8, dev: {1: np.float64(0.5735714285714286), 5: np.float64(0.6278571428571429), 10: np.float64(0.6338432539682539)}


Results for Ensemble BM25 with all features and different models, k1=1.97, b=0.84, train: {1: np.float64(0.57721932622734), 5: np.float64(0.6279986514173086), 10: np.float64(0.6334984791395746)}
Results for Ensemble BM25 with all features and different models, k1=1.97, b=0.84, dev: {1: np.float64(0.5792857142857143), 5: np.float64(0.63375), 10: np.float64(0.6390765306122449)}


Results for Ensemble BM25 with all features and different models, k1=1.95, b=0.83, train: {1: np.float64(0.5770637205321715), 5: np.float64(0.6279753105630332), 10: np.float64(0.6334749530404241)}
Results for Ensemble BM25 with all features and different models, k1=1.95, b=0.83, dev: {1: np.float64(0.5807142857142857), 5: np.float64(0.6342619047619048), 10: np.float64(0.6398863378684807)}


In [65]:
weights = {
    'title': 1.0,
    'abstract': 2.0,
    'last_names': 0.3,
    'journal': 0.1,
    'publish_year': 0
}
df_query_train_ensemble_single = experiment_ensemble_single_feature(df_collection, df_query_train, weights=weights, k1=1.2, b=0.74)
df_query_dev_ensemble_single = experiment_ensemble_single_feature(df_collection, df_query_dev, weights=weights)
evaluate_experiment(df_query_train_ensemble_single,df_query_dev_ensemble_single, "Ensemble BM25 with single features")

Results for Ensemble BM25 with single features, train: {1: np.float64(0.5799424258927877), 5: np.float64(0.6291760678440831), 10: np.float64(0.6342819105909434)}
Results for Ensemble BM25 with single features, dev: {1: np.float64(0.5878571428571429), 5: np.float64(0.6388095238095237), 10: np.float64(0.6432848639455782)}


{1: np.float64(0.5878571428571429),
 5: np.float64(0.6388095238095237),
 10: np.float64(0.6432848639455782)}